In [1]:
# pip install langchain langchain_community python-dotenv

In [17]:
# Langsmith tracing
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("langsmith_api")

In [18]:
from langchain_community.chat_models import ChatOllama

local_llm = "llama3"
model = ChatOllama(model=local_llm, temperature=0.8)

from langchain_core.messages import HumanMessage

print(model.invoke([HumanMessage(content="Oi, eu sou o Bruno!")]))
print(model.invoke([HumanMessage(content="Qual é o meu nome?")]))

# O LLM não guarda o estado da conversa

content='Olá Bruno!\n\nBem-vindo ao chat! Como vai você?' response_metadata={'model': 'llama3', 'created_at': '2024-07-03T16:45:28.95237Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 7778028400, 'load_duration': 2767973700, 'prompt_eval_count': 17, 'prompt_eval_duration': 1719351000, 'eval_count': 16, 'eval_duration': 3287017000} id='run-d15d4eeb-0e6c-4f81-bd54-f50cab91541b-0'
content='Peço desculpas, mas como sou uma inteligência artificial, não tenho conhecimento sobre você ou seu nome. Eu sou um modelo de linguagem treinado para fornecer informações e responder perguntas, mas não tenho acesso a informações pessoais sobre indivíduos específicos.' response_metadata={'model': 'llama3', 'created_at': '2024-07-03T16:45:43.7097905Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 12723086000, 'load_duration': 2671400, 'prompt_eval_count': 11, 'prompt_eval_duration': 900809

In [19]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Oi, eu sou o Bruno!"),
        AIMessage(content="Olá Bruno! Prazer em conhecer você!"),
        HumanMessage(content="Qual é o meu nome?"),
    ]
)

# Com um contexto ele consegue responder sobre informações da própria conversa

AIMessage(content='Bruno!', response_metadata={'model': 'llama3', 'created_at': '2024-07-03T16:45:56.3593813Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 3406059600, 'load_duration': 1616500, 'prompt_eval_count': 39, 'prompt_eval_duration': 2878196000, 'eval_count': 4, 'eval_duration': 524486000}, id='run-1834c5be-ac84-46ae-9da4-e7d21794cfec-0')

In [74]:
from langchain_community.chat_message_histories import ChatMessageHistory
# A classe ChatMessageHistory indica que o histórico será guardado em memória
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

# store será utilizado para guardar informações de múltiplas conversas/chats


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Função que verifica a existência de uma conversa anterior ou se é uma nova

with_message_history = RunnableWithMessageHistory(model, get_session_history)

# with_message_history será usado no lugar do model.invoke já que, passando a função get_session_history que retorna um objeto ChatMessageHistory
# ChatMessageHistory tem o formato de Lista de BaseMessage

In [68]:
config = {"configurable": {"session_id": "abc44"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Brunoasd")],
    config=config,
)

print(response.content)

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

print(response.content)

Nice to meet you, Brunoasd! What brings you here today? Do you have a specific question or topic you'd like to discuss? I'm all ears!
According to our conversation so far, your name is Brunoasd!


In [73]:
# Adicionando prompt templates

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente de chat que só consegue falar as coisas no diminutivo. Responda tudo em português e no diminutivo.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [70]:
response = chain.invoke({"messages": [HumanMessage(content="Oi eu sou o Bruno!")]})

print(response.content)

Oi, Brunozinho! Como você está hoje?


In [76]:
# Podemos juntar o prompt template com o chat history

store = {}

# store será utilizado para guardar informações de múltiplas conversas/chats


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable": {"session_id": "teste1234"}}

response = with_message_history.invoke(
    [HumanMessage(content="Olá, eu sou o Bruno!")],
    config=config,
)

print(response.content)

response = with_message_history.invoke(
[HumanMessage(content="Qual é o seu maior pesadelo?")],
    config=config,
)

print(response.content)

Olá, você é um Brunozinho!
Meu maior pesadinho é não poder responder às perguntinhas com rapidez e inteligência!


In [77]:
# Podemos deixar o prompt template ainda mais complexo


prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente de chat. Responda todas as mensagens em {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

response = chain.invoke(
    {"messages": [HumanMessage(content="Oi eu sou o Bruno!")], "language": "Japonês"}
)

print(response.content)

(Konnichiwa, Bruno-san!)


In [85]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc111"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Oi eu sou o Bruno!")], "language": "Japonês"},
    config=config,
)

print(response.content)

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Qual é o meu nome?")], "language": "Espanhol"},
    config=config,
)

print(response.content)

Mata ne, Bruno-san! Sore de, kono chatto wa Bruno to iu namae o shirimasu!
¡Bruno es tu nombre!


In [88]:
# Para dar a experiência mais dinâmica no chat, podemos printar os tokens na medida que são gerados pelo LLM com streaming

config = {"configurable": {"session_id": "abc15"}}
for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="Tell me a joke!")],
        "language": "Japonês",
    },
    config=config,
):
    print(r.content, end="")

😊

Gakusei no nan ka o shite mita no ka?

Aru gakusei wa jibun no seikō o hanasu to, sensei ni "Watashi wa yūjī ni naru" to itta!

Hahaha, yoroshiku onegaishimasu! (I hope you enjoyed it!)